# GridSarchCVとPipelineについて勉強

In [9]:
# データ準備
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
# 欠損値をdrop
df = df.dropna()
X = df.loc[:, (df.columns != 'survived') & (df.columns != 'alive')]
# 質的変数はダミー変数に変換
X = pd.get_dummies(X, drop_first=True)
y = df['survived']

In [10]:
# モデル準備
from sklearn.svm import SVC
model = SVC(random_state=0)

In [11]:
# ハイパーパラメータの組み合わせディクショナリーを作成
# keyにはモデルの引数名，valueにはそのハイパーパラメータの値のリスト
svm_param_grid = {'kernel': ['linear', 'rbf'],
            'C': [2**i for i in range(-2, 3)]}

In [12]:
# cvオブジェクトを作成
# 分類タスクであれば RepeatedStratifiedKFold クラスを使うのがよい(回帰の場合は RepeatedKFold ).
# 指定したk-Foldをさらに複数回実行します(Repeated).
# クラスの偏りが無いようにもとのクラスの割合をキープしてデータを分割(Stratified).
# 今回は5-Foldを３回繰り返すようにする
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)

In [13]:
# GridSarchCVオブジェクト作成&実行
"""
NOTE: 
GridSearchCVは, 標準化などの前処理を組み込むことができない.
標準化のような”学習データと検証データを分けてから実行するような前処理”は,
CVの処理の中に組み込む必要がある
"""
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=svm_param_grid, scoring='accuracy', cv=cv)
grid_search.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=0),
             estimator=SVC(random_state=0),
             param_grid={'C': [0.25, 0.5, 1, 2, 4],
                         'kernel': ['linear', 'rbf']},
             scoring='accuracy')

In [14]:
print(grid_search.best_params_, grid_search.best_score_)

{'C': 0.25, 'kernel': 'linear'} 0.7618118118118118


In [15]:
# PipelineとGridSearchCVを組み合わせる
# 前処理 -> GridSearchCV ができる

In [16]:
# Pipelineオブジェクト作成
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC(probability=True, random_state=0))])

In [17]:
# param_dict作成
# Pipelineの“どの処理のパラメータなのか”がわかる必要あり.
# ”処理名__パラメータ名”をキーにする
svm_param_grid = {'model__kernel': ['linear', 'rbf'], 
                    'model__C': [2**i for i in range(-2, 3)]}

In [18]:
# GridSearchCVを実行
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
grid_search = GridSearchCV(estimator=pipeline, param_grid=svm_param_grid, scoring='accuracy', cv=cv)
grid_search.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=0),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        SVC(probability=True,
                                            random_state=0))]),
             param_grid={'model__C': [0.25, 0.5, 1, 2, 4],
                         'model__kernel': ['linear', 'rbf']},
             scoring='accuracy')

# Pipeline+GridSearchCVを いちからコーディング

In [19]:
# データ準備
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
# 欠損値をdrop
df = df.dropna()
X = df.loc[:, (df.columns != 'survived') & (df.columns != 'alive')]
# 質的変数はダミー変数に変換
X = pd.get_dummies(X, drop_first=True)
y = df['survived']

In [20]:
# Pipelineオブジェクト作成
# scaler, model, pipelineの準備
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC(probability=True, random_state=0))])

In [21]:
# param_dict作成
# Pipelineの“どの処理のパラメータなのか”がわかる必要あり.
# ”処理名__パラメータ名”をキーにする
svm_param_grid = {'model__kernel': ['linear', 'rbf'], 
                    'model__C': [2**i for i in range(-2, 3)]}

In [22]:
# GridSearchCVを実行
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
grid_search = GridSearchCV(estimator=pipeline, param_grid=svm_param_grid, scoring='accuracy', cv=cv)
grid_search.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=0),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        SVC(probability=True,
                                            random_state=0))]),
             param_grid={'model__C': [0.25, 0.5, 1, 2, 4],
                         'model__kernel': ['linear', 'rbf']},
             scoring='accuracy')

In [23]:
print(grid_search.best_params_, grid_search.best_score_)

{'model__C': 0.5, 'model__kernel': 'rbf'} 0.752729044834308
